In [2]:
import torch 
from transformers import BertTokenizer, BertModel 

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [4]:
text1 = "These are just a few ways that one might use the random sentence generator for their benefit."
text2 = "There exist few advantages of using random sentence generator in real life."

text1 = "[CLS]" + text1 + "[SEP]"
text2 = "[CLS]" + text2 + "[SEP]" 

texts = [text1, text2]

tokenize_text1 = tokenizer.tokenize(text1)
tokenize_text2 = tokenizer.tokenize(text2)

indexed_token1 = tokenizer.convert_tokens_to_ids(tokenize_text1)
indexed_token2 = tokenizer.convert_tokens_to_ids(tokenize_text2)

In [8]:
segment1 = [1] * len(tokenize_text1)
segment2 = [1] * len(tokenize_text2)

In [9]:
indexed_token1 = torch.tensor([indexed_token1])
indexed_token2 = torch.tensor([indexed_token2])

segment1 = torch.tensor([segment1])
segment2 = torch.tensor([segment2])

In [15]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [16]:
with torch.no_grad():
    output1 = model(indexed_token1, segment1)
    output2 = model(indexed_token2, segment2)

    hidden1 = output1[2]
    hidden2 = output2[2]

In [17]:
hidden1[0].size()

torch.Size([1, 20, 768])

In [19]:
hidden2[0].size()

torch.Size([1, 15, 768])

In [20]:
"""
sentence vecrtorisation
"""

token_vecs1 = hidden1[-2][0]
token_vecs2 = hidden2[-2][0]

sentence_embedding1 = torch.mean(token_vecs1, dim=0)
sentence_embedding2 = torch.mean(token_vecs2, dim=0)

In [21]:
sentence_embedding1.size()

torch.Size([768])

In [22]:
sentence_embedding2.size()

torch.Size([768])

In [26]:
"""
Finding the consine similarity of these two question
"""
from scipy.spatial.distance import cosine

smilarity_score = 1- cosine(sentence_embedding1, sentence_embedding2)

In [27]:
smilarity_score

0.8352345824241638